<a href="https://colab.research.google.com/github/Delaunay-I/deep_learning/blob/main/LeNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import the necessary packages
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras import backend as K

It’s common to see the number of CONV filters learned increase in deeper layers of the network.

In [ ]:
class LeNet:
	def build(numChannels, imgRows, imgCols, numClasses,
		activation="relu", weightsPath=None):
		# initialize the model
		model = Sequential()
		inputShape = (imgRows, imgCols, numChannels)
		# if we are using "channels first", update the input shape
		if K.image_data_format() == "channels_first":
			inputShape = (numChannels, imgRows, imgCols)
   
   	# define the first set of CONV => ACTIVATION => POOL layers
		model.add(Conv2D(20, 5, padding="same",
			input_shape=inputShape, activation=activation))
		model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
  
  	# define the second set of CONV => ACTIVATION => POOL layers
		model.add(Conv2D(50, 5, padding="same", activation=activation))
		model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
  
  	# define the first FC => ACTIVATION layers
		model.add(Flatten())
		model.add(Dense(500, activation= activation))
		# define the second FC layer
		model.add(Dense(numClasses))
		# lastly, define the soft-max classifier
		model.add(Activation("softmax"))
  
  	# if a weights path is supplied (inicating that the model was
		# pre-trained), then load the weights
		if weightsPath is not None:
			model.load_weights(weightsPath)
		# return the constructed network architecture
		return model

In [ ]:
from sklearn.model_selection import train_test_split
from keras.datasets import mnist
from keras.optimizers import SGD
from keras.utils import np_utils
import numpy as np
import cv2

In [ ]:
print("[INFO] downloading MNIST...")
((X_train, y_train),(X_test, y_test)) = mnist.load_data()

# reshape the dataset into a matrix with format of:
# num_samples x depth x rows x columns
X_train = X_train.reshape((X_train.shape[0], 28, 28, 1))
X_test = X_test.reshape((X_test.shape[0], 28, 28, 1))

# Scale the data to range [0, 1]
X_train = X_train.astype("float32") / 255.0
X_test = X_test.astype("float32") / 255.0

y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)

[INFO] downloading MNIST...
11490434/11490434 [==============================] - 1s 0us/step


In [ ]:
# Initialize optimizer and model
print("[INFO] compiling model...")
opt = SGD(learning_rate=0.01)
model = LeNet.build(numChannels=1, imgRows = 28, imgCols = 28, numClasses = 10)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=["accuracy"])

[INFO] compiling model...


In [ ]:
print("[INFO] training...")
model.fit(x=X_train, y=y_train, batch_size=128, epochs=20, verbose=1, validation_data=(X_test, y_test))

[INFO] training...
Epoch 1/20
469/469 [==============================] - 12s 7ms/step - loss: 0.9499 - accuracy: 0.7480 - val_loss: 0.3393 - val_accuracy: 0.8993
Epoch 2/20
469/469 [==============================] - 4s 9ms/step - loss: 0.2555 - accuracy: 0.9238 - val_loss: 0.1981 - val_accuracy: 0.9390
Epoch 3/20
469/469 [==============================] - 5s 11ms/step - loss: 0.1808 - accuracy: 0.9460 - val_loss: 0.1451 - val_accuracy: 0.9574
Epoch 4/20
469/469 [==============================] - 4s 9ms/step - loss: 0.1424 - accuracy: 0.9581 - val_loss: 0.1198 - val_accuracy: 0.9626
Epoch 5/20
469/469 [==============================] - 3s 7ms/step - loss: 0.1190 - accuracy: 0.9650 - val_loss: 0.0988 - val_accuracy: 0.9704
Epoch 6/20
469/469 [==============================] - 4s 8ms/step - loss: 0.1011 - accuracy: 0.9698 - val_loss: 0.0927 - val_accuracy: 0.9699
Epoch 7/20
469/469 [==============================] - 3s 6ms/step - loss: 0.0892 - accuracy: 0.9736 - val_loss: 0.0740 - val_ac

In [ ]:
print("[INFO] evaluating...")
(loss, accuracy) = model.evaluate(x=X_test, y=y_test, batch_size=128, verbose=1)
print(f"[INFO] accuracy: {accuracy * 100}")

[INFO] evaluating...
79/79 [==============================] - 0s 4ms/step - loss: 0.0429 - accuracy: 0.9856
[INFO] accuracy: 98.55999946594238


In [ ]:
model.save_weights("./LeNet.hdf5", overwrite=True)